## Demo in action!

## Load Libraries
We will loads some libraries needed by the application. 
>The BrAPI Helper will also load some libraries for us.

In [ ]:
library(QBMS)
suppressMessages(library(tidyverse))
library(patchwork)
library(RColorBrewer)

## Retrieve genotype data from Gigwa
Interacting with the genotype database is done via the `geno_provider` variable. This variable is created by the BrAPI helper tool upon sucessful login.

For this excersise we created a genotype database entry called `rtassel_demo` and stored a HapMap formatted file as `rtassel_hmp`.

We will use the `geno_provider` variable to access and retrieve this data.


In [ ]:
# We can explore databases "entries" with
geno_provider$gigwa_list_dbs() |>
     as_tibble() |>
     rename(available_databases = value)

In [ ]:
# Now we setup access for rtassel_demo
geno_provider$gigwa_set_db("rtassel_demo")

In [ ]:
# We can list projects
geno_provider$gigwa_list_projects()

In [ ]:
# Now we setup access for rtassel_demo->rtassel_hmp
geno_provider$gigwa_set_project("rtassel_hmp")

In [ ]:
# We get a list of the samples available for extraction. 
# In our case these samples match the phenotype data.
geno_samples <- geno_provider$gigwa_get_samples()

In [ ]:
# For demo purposes we will use GIGWAs built in filtering capabilities 
# to reduce the number of SNPs retrieved.
# We will also restrict samples.
genoDataFromGigwa <- geno_provider$gigwa_get_variants(max_missing = 0.2,
                                                      min_maf = 0.35,
                                                      samples=geno_samples[1:30])

In [ ]:
# We take a quick look at the retrieved data
head(genoDataFromGigwa)

### Genotype Data Visualization
We will create a heatmap for the retrieved genotypes stored on the `genoDataFromGigwa` variable.

In [ ]:
# We will remove the metadata columns 1-4 and only keep the genotype columns
genoData <- genoDataFromGigwa |>
  select(-c(1:4)) |>
  as.matrix()

heatmap(genoData, col=brewer.pal(4, "PuRd"))

## Retrieve phenotype data from BMS

Interacting with the phenotype database is done via the `pheno_provider` variable. As a reminder, this variable is created by the BrAPI helper tool upon sucessful login.

For this excersise we created a `Maize` breeding program called `rTASSEL Testbed` with a breeding trial called `rTASSEL_Demo_wHapMap`. Inside this trial we created phenotype dataset for a dummy environment called `Environment Number 1`

We will use the `pheno_provider` variable to access and retrieve this data.

In [ ]:
pheno_provider$list_crops() |>
     as_tibble() |>
     rename(available_crops = value)

In [ ]:
pheno_provider$set_crop("maize")

In [ ]:
pheno_provider$list_programs()

In [ ]:
pheno_provider$set_program("rTASSEL Testbed")

In [ ]:
pheno_provider$list_trials()

In [ ]:
pheno_provider$set_trial("rTASSEL_Demo_wHapMap")

In [ ]:
pheno_provider$list_studies()

In [ ]:
pheno_provider$set_study(study_name = "rTASSEL_Demo_wHapMap Environment Number 1")

In [ ]:
# select Corn Ear Height (EH_M_cm) and Diameter (EDia_M_cm) 
# and days to pollination (Dpoll)
pheno_df <- pheno_provider$get_study_data() |>
    select(germplasmName, EH_M_cm, EDia_M_cm, Dpoll) |>  
    mutate(across(c(EH_M_cm, EDia_M_cm, Dpoll), as.numeric))

### Phenotype Data Visualization
We will create a histogram for each of the retrieve traits, Corn Ear Height (EH_M_cm) and Diameter (EDia_M_cm).

In [ ]:
# Create the histogram for EH_M_cm
fig_EH <- pheno_df |> ggplot() +
  aes(EH_M_cm) +
  geom_histogram(fill = "blue", bins = 30) +
  labs(title = "Ear Height (cm)") +
  theme_minimal()

# Create the histogram for EDia_M_cm
fig_EDia <- pheno_df |> ggplot() +
  aes(EDia_M_cm) +
  geom_histogram(fill = "red", bins = 30) +
  labs(title = "Ear Diameter (cm)") +
  theme_minimal()

# Create the histogram for Dpoll
fig_Dpoll <- pheno_df |> ggplot() +
  aes(Dpoll) +
  geom_histogram(fill = "green", bins = 30) +
  labs(title = "Days to Pollination") +
  theme_minimal()

# Combine the plots
combined_plots <- fig_EH / fig_EDia / fig_Dpoll 

# Print the combined plot
combined_plots

## The END

At this point, you have already loaded and visualized data directly from two databases. From this point on, you can use the data to perform various analyses.

You can explore other templates that have been created for this purpose by looking at our template repository.
